In [1]:
import os, sys, logging

import numpy as np
import matplotlib.pyplot as plt

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from data.lorenz import Lorenz
from data.double_well import DoubleWellStochastic
from transform.transfer_operator import * 
from misc.utils import eigs_sort #, abs_eig_sort, real_eig_sort

# Keep 3 digits of accuracy, and avoid scientific notation when possible.
np.set_printoptions(precision=3, suppress=True, threshold=100) 

from scipy.sparse.linalg import eigs, eigsh
from matplotlib.animation import FuncAnimation
from matplotlib import rc, lines
from IPython.display import HTML
rc('animation', html='html5')
rc('savefig', dpi=300)


ImportError: cannot import name 'EulerMaruyama' from 'data.integrate' (/Users/Jesse/Projects/masters-thesis/code/data/integrate.py)

In [ ]:
double_well_system = DoubleWellStochastic(temperature=0.32, step_size=.001)
transfer_operator = TransferOperator(labeling_method="uniform", n_delays=0, embedding_dim=1)


In [ ]:
# T = 5 x 10^5
# dt = 10e-3
# => n_steps = 5 x 10^8
# \sqrt{2\beta^-1} = 0.8a
#double_well_samples = double_well_system.gen_data_or_load(n_steps=1000000)
double_well_samples = double_well_system.gen_data(n_steps=1000000)
double_well_system.save(double_well_samples)
double_well_system.pdf(double_well_samples)
# _ = double_well_system.get_boltzmann_dist(double_well_samples)
_ = double_well_system.get_boltzmann_dist_exact()

In [ ]:
xs = np.arange(-2, 2, .01)
get_potential = lambda x: np.square(np.square(x)-1.) + 1
potential = get_potential(xs)

fig, ax = plt.subplots()
xdata, ydata = [], []
ln, = plt.plot([], [], 'ro')
plt.plot(xs, potential) 

ax.set_ylim(-0.5,9)
ax.set_xticks([-2, -1, 0, 1, 2])
#ax.set_yticks([])
ax.set_frame_on(False)
ax.get_xaxis().tick_bottom()
plt.tight_layout()

ax.axes.get_yaxis().set_visible(False)
xmin, xmax = ax.get_xaxis().get_view_interval()
ax.add_artist(lines.Line2D((xmin, xmax), (0, 0), color='black', linewidth=2))
ax.spines['bottom'].set_position(('data', 0))

def update(frame):
    x = double_well_samples[40000 + 40 * frame]
    xdata = [x, x]
    ydata = ([0,  get_potential(x)])
    ln.set_data(xdata, ydata)
    return ln,

ani = FuncAnimation(fig, update, frames=1500, interval=10, blit=False)
ani

In [ ]:
xs = np.arange(-5, 5, .01)
get_potential = lambda x: np.square(np.square(x)-1.) + 1
potential = get_potential(xs)

fig, ax = plt.subplots()
xdata, ydata = [], []
ln, = plt.plot([], [], 'ro')
plt.plot(xs, potential) 
time_text = ax.text(10, 50,'R', fontsize=72, fontweight="bold")

ax.set_ylim(-0.5,100)
ax.set_xlim(-20,20)

#ax.set_yticks([])
ax.set_xticks([0])
ax.set_xticklabels([""])

ax.set_frame_on(False)
ax.get_xaxis().tick_bottom()
plt.tight_layout()

ax.axes.get_yaxis().set_visible(False)
xmin, xmax = ax.get_xaxis().get_view_interval()
ax.add_artist(lines.Line2D((xmin, xmax), (0, 0), color='black', linewidth=2))
ax.spines['bottom'].set_position(('data', 0))

def update(frame):
    xdata = double_well_samples[40000 + 40 * frame]
    ydata = (xdata)
    ln.set_data(xdata, ydata)
    
    label = "L"
    xpos = -10
    if xdata > 0:
        label = "R"
        xpos = 10
    
    time_text.set_text(label)
    time_text.set_x(xpos)
    return ln, time_text

ani = FuncAnimation(fig, update, frames=1500, interval=10, blit=False)
ani

In [ ]:
np.random.seed(123)

logging.basicConfig(level=logging.INFO)


downsampler = DownSampler(transfer_operator, 1000)
trans_matrix = downsampler.fit_trans_matrix(double_well_samples)
invariant_dist = transfer_operator.get_invariant_dist(trans_matrix)
rev_trans_matrix = transfer_operator.get_reversible_trans_matrix(trans_matrix, invariant_dist)

In [ ]:
spectrum_1, eig_vecs_1 = eigs_sort(trans_matrix.T, 50, which='LR')

stable_dist = np.abs(eig_vecs_1[:,0])
min_x = np.amin(double_well_samples, axis=0)
max_x = np.amax(double_well_samples, axis=0)
step = (max_x-min_x) / stable_dist.size

plt.plot(np.arange(min_x, max_x-step+.0001, step), stable_dist)
plt.title("Stationary distribution of the (transpose) transfer matrix.")
plt.ylabel("P(x)")
plt.xlabel("x")

In [ ]:
plt.plot(np.arange(1,51), spectrum_1) 
print(spectrum_1)
plt.title("Eigenvalue spectrum of the transfer matrix.")
plt.xlabel("Eigenvalue index")
plt.ylabel("Absolute value.")

In [ ]:
fig, ax = plt.subplots()
xdata, ydata = [], []
ln, = plt.plot(np.arange(min_x, max_x-step+.001, step), eig_vecs_1[:, 0])

ax.set_xlabel("x")    
ax.set_ylabel("y")
ax.set_ylim((-.25, .25))

def update(frame):
    ax.set_title("Eigenfunction {} of the nonreversibilized transfer matrix. Eigenvalue={}.".format(frame+1, spectrum_1[frame]))
    ln.set_data(np.arange(min_x, max_x-step+0.001, step), eig_vecs_1[:, frame])
    return ln,

anim = FuncAnimation(fig, update, frames=4, blit=False, interval=1000)

HTML(anim.to_html5_video())

In [ ]:
spectrum, eig_vecs = eigs(rev_trans_matrix, 50, which='LR')
print(spectrum)


spectrum[spectrum ==np.inf] = 0
spectrum[spectrum ==np.NaN] = 0
plt.plot(np.arange(1,51), spectrum)


In [ ]:
stable_dist_2  = eig_vecs[:,0]
stable_dist_3  = eig_vecs[:,1]
 
plt.title("First two eigenfunctions of the reversibilized transfer matrix.")
plt.xlabel("x")
plt.ylabel("y")
plt.plot(np.arange(min_x, max_x, step), stable_dist_2)
plt.plot(np.arange(min_x, max_x, step), stable_dist_3)
plt.show()



#double_well_system.pdf(reconstructed_samples[:,:1])
#double_well_system.get_boltzmann_dist(reconstructed_samples[:,:1])

In [ ]:
fig, ax = plt.subplots()
xdata, ydata = [], []
ln, = plt.plot(np.arange(min_x, max_x, step), eig_vecs[:, 0])

ax.set_xlabel("x")    
ax.set_ylabel("y")
ax.set_ylim((-.25, .25))

def update(frame):
    ax.set_title("Eigenfunction {} of the reversibilized transfer matrix. Eigenvalue={}.".format(frame+1, spectrum[frame]))
    ln.set_data(np.arange(min_x, max_x, step), eig_vecs[:, frame])
    return ln,

anim = FuncAnimation(fig, update, frames=50, blit=False, interval=350)

HTML(anim.to_html5_video())

![figB](./images/figB.png)

In [ ]:
double_well_samples_2 = double_well_system.gen_data(n_steps=100000)

downsampler_2 = DownSampler(transfer_operator, 1000)
double_well_samples_split_2 = downsampler_2.downsample(double_well_samples_2)

def plot_t_imp_scaling(time_series, eigval_idx, n_partitionings_list=[2, 10, 100], taus=range(1, 17, 4), transition_time=1):
    plt.title("Scaling of $t$ with $\\tau$ and $n_p$")
    for n_partitionings in n_partitionings_list:
        t_imps = np.zeros(len(taus))
        for i, tau in tqdm(enumerate(taus), desc="computing t_imp"):
            t_imps[i] = downsampler_2.get_t_imp(time_series, eigval_idx, n_partitionings, tau, transition_time)
            
        plt.plot(taus, t_imps, label="$n_p = {}$".format(n_partitionings))

    plt.legend()
    plt.show()
    
plot_t_imp_scaling(double_well_samples_split_2, 1)
    
    
    

---

In [ ]:
lorenz_system = Lorenz()
lorenz_samples = lorenz_system.gen_data() 


In [ ]:
lorenz_system.plot_trajectory(lorenz_samples, "xy")

In [ ]:
lorenz_transfer = TransferOperator()
lorenz_reconstructed = lorenz_transfer.fit_transform(
    lorenz_samples[:, 0:1],
    embedding_dim_range=np.arange(1,5),
    delay_range=np.arange(1,20), 
    delay_clusters_range=np.arange(100,101), 
    dim_clusters_range=np.arange(100,101)
)

In [ ]:
lorenz_system.plot_trajectory(lorenz_reconstructed, "xy")

In [ ]:
1000000